## AAM / Machine Learning - Atividade 03/04 - Analise de Score

## *Ciro Mora* - RA: 111310

### O dataset a ser analisado é uma descrição de todos os arremesos a cesta de um jogador de basquete (https://www.kaggle.com/c/kobe-bryant-shot-selection), nesse caso Kobe Bryant.  
  
Iniciamos com importações padrao para iniciar as análises.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

import warnings
warnings.filterwarnings('ignore')

from numba import jit

As proximas celulas são focadas em pegar um overview do dataset.  
A funcao de eval abaixo é uma função para acelerar processamento das analises e regressoes

In [2]:
@jit
def eval_gini(y_true, y_prob):
    """
    Original author CPMP : https://www.kaggle.com/cpmpml
    In kernel : https://www.kaggle.com/cpmpml/extremely-fast-gini-computation
    """
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2 * gini / (ntrue * (n - ntrue))
    return gini

In [3]:
dfTotal = pd.read_csv('../input/avaliacao.csv')

In [4]:
headNumber = 5
print(f'Dataset de treino - Primeiras {headNumber} linhas')
#display(dfTotal.head(headNumber))

Dataset de treino - Primeiras 5 linhas


,action_type,combined_shot_type,game_event_id,game_id,lat,loc_x,loc_y,lon,minutes_remaining,period,playoffs,season,seconds_remaining,shot_distance,shot_made_flag,shot_type,shot_zone_area,shot_zone_basic,shot_zone_range,team_id,team_name,game_date,matchup,opponent,shot_id
0,Jump Shot,Jump Shot,10,20000012,33.9723,167,72,-118.1028,10,1,0,2000-01,27,18,NaN,2PT Field Goal,Right Side(R),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,1
1,Jump Shot,Jump Shot,12,20000012,34.0443,-157,0,-118.4268,10,1,0,2000-01,22,15,0.0,2PT Field Goal,Left Side(L),Mid-Range,8-16 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,2
2,Jump Shot,Jump Shot,35,20000012,33.9093,-101,135,-118.3708,7,1,0,2000-01,45,16,1.0,2PT Field Goal,Left Side Center(LC),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,3
3,Jump Shot,Jump Shot,43,20000012,33.8693,138,175,-118.1318,6,1,0,2000-01,52,22,0.0,2PT Field Goal,Right Side Center(RC),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,4
4,Driving Dunk Shot,Dunk,155,20000012,34.0443,0,0,-118.2698,6,2,0,2000-01,19,0,1.0,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,5


In [5]:
print('Dataset - Estatistica descritiva')
#display(dfTotal.describe())

Dataset - Estatistica descritiva


,game_event_id,game_id,lat,loc_x,loc_y,lon,minutes_remaining,period,playoffs,seconds_remaining,shot_distance,shot_made_flag,team_id,shot_id
count,30697.000000,3.069700e+04,30697.000000,30697.000000,30697.000000,30697.000000,30697.000000,30697.000000,30697.000000,30697.000000,30697.000000,25697.000000,3.069700e+04,30697.000000
mean,249.190800,2.476407e+07,33.953192,7.110499,91.107535,-118.262690,4.885624,2.519432,0.146562,28.365085,13.437437,0.446161,1.610613e+09,15349.000000
std,150.003712,7.755175e+06,0.087791,110.124578,87.791361,0.110125,3.449897,1.153665,0.353674,17.478949,9.374189,0.497103,0.000000e+00,8861.604943
min,2.000000,2.000001e+07,33.253300,-250.000000,-44.000000,-118.519800,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.610613e+09,1.000000
25%,110.000000,2.050008e+07,33.884300,-68.000000,4.000000,-118.337800,2.000000,1.000000,0.000000,13.000000,5.000000,0.000000,1.610613e+09,7675.000000
50%,253.000000,2.090035e+07,33.970300,0.000000,74.000000,-118.269800,5.000000,3.000000,0.000000,28.000000,15.000000,0.000000,1.610613e+09,15349.000000
75%,368.000000,2.960047e+07,34.040300,95.000000,160.000000,-118.174800,8.000000,3.000000,0.000000,43.000000,21.000000,1.000000,1.610613e+09,23023.000000
max,659.000000,4.990009e+07,34.088300,248.000000,791.000000,-118.021800,11.000000,7.000000,1.000000,59.000000,79.000000,1.000000,1.610613e+09,30697.000000


In [6]:
print('Dataset - Sumário das Features')
print(dfTotal.info())

Dataset - Sumário das Features
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30697 entries, 0 to 30696
Data columns (total 25 columns):
action_type           30697 non-null object
combined_shot_type    30697 non-null object
game_event_id         30697 non-null int64
game_id               30697 non-null int64
lat                   30697 non-null float64
loc_x                 30697 non-null int64
loc_y                 30697 non-null int64
lon                   30697 non-null float64
minutes_remaining     30697 non-null int64
period                30697 non-null int64
playoffs              30697 non-null int64
season                30697 non-null object
seconds_remaining     30697 non-null int64
shot_distance         30697 non-null int64
shot_made_flag        25697 non-null float64
shot_type             30697 non-null object
shot_zone_area        30697 non-null object
shot_zone_basic       30697 non-null object
shot_zone_range       30697 non-null object
team_id               30697 no

In [7]:
print(f'Dataset tem {dfTotal.shape[0]} linhas por {dfTotal.shape[1]} colunas ({dfTotal.shape[0] * dfTotal.shape[1]} celulas)')

Dataset tem 30697 linhas por 25 colunas (767425 celulas)


## Remocao de colunas dadas como desnecessarias

Ao analisar o Database, as descricoes de algumas colunas nao pareciam ter qualquer relacao com o resultado (score ou nao), sendo Ids de jogos, nome do time, periodo do jogo, se o jogo era do tipo playoff ou nao. Os outros dados serao analisados por RFE para remocao.
colunas removidas -   
game_event_id  
game_id  
team_id  
team_name  
period  
playoffs  
shot_id  
Mais colunas poderiam ser removidas, porem achei importante executar com todos os dados oferecidos primeiro

In [8]:
dfTotal = dfTotal.drop(columns=['game_event_id',  
'game_id',
'team_id',  
'team_name',
'period',
'playoffs',   
'shot_id'])

### As colunas minutes e seconds desempenham uma funcao parecida, entao irei juntar elas numa so, convertendo para soma dos segundos.
### A coluna shot made flag, que contem dados vazios, sao os dados para comparacao de resultado. Eles serao removidos

In [9]:
dfComplete = dfTotal.dropna(subset=['shot_made_flag'])
dfComplete.minutes_remaining = dfComplete.minutes_remaining*60
dfComplete.seconds_remaining = pd.to_numeric(dfComplete.seconds_remaining)
dfComplete

,action_type,combined_shot_type,lat,loc_x,loc_y,lon,minutes_remaining,season,seconds_remaining,shot_distance,shot_made_flag,shot_type,shot_zone_area,shot_zone_basic,shot_zone_range,game_date,matchup,opponent
1,Jump Shot,Jump Shot,34.0443,-157,0,-118.4268,600,2000-01,22,15,0.0,2PT Field Goal,Left Side(L),Mid-Range,8-16 ft.,2000-10-31,LAL @ POR,POR
2,Jump Shot,Jump Shot,33.9093,-101,135,-118.3708,420,2000-01,45,16,1.0,2PT Field Goal,Left Side Center(LC),Mid-Range,16-24 ft.,2000-10-31,LAL @ POR,POR
3,Jump Shot,Jump Shot,33.8693,138,175,-118.1318,360,2000-01,52,22,0.0,2PT Field Goal,Right Side Center(RC),Mid-Range,16-24 ft.,2000-10-31,LAL @ POR,POR
4,Driving Dunk Shot,Dunk,34.0443,0,0,-118.2698,360,2000-01,19,0,1.0,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,2000-10-31,LAL @ POR,POR
5,Jump Shot,Jump Shot,34.0553,-145,-11,-118.4148,540,2000-01,32,14,0.0,2PT Field Goal,Left Side(L),Mid-Range,8-16 ft.,2000-10-31,LAL @ POR,POR
6,Layup Shot,Layup,34.0443,0,0,-118.2698,480,2000-01,52,0,1.0,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,2000-10-31,LAL @ POR,POR
8,Jump Shot,Jump Shot,33.9363,-65,108,-118.3348,360,2000-01,12,12,1.0,2PT Field Goal,Left Side(L),In The Paint (Non-RA),8-16 ft.,2000-10-31,LAL @ POR,POR
9,Running Jump Shot,Jump Shot,33.9193,-33,125,-118.3028,180,2000-01,36,12,0.0,2PT Field Goal,Center(C),In The Paint (Non-RA),8-16 ft.,2000-10-31,LAL @ POR,POR
10,Jump Shot,Jump Shot,33.8063,-94,238,-118.3638,60,2000-01,56,25,0.0,3PT Field Goal,Left Side Center(LC),Above the Break 3,24+ ft.,2000-10-31,LAL @ POR,POR
11,Jump Shot,Jump Shot,33.9173,121,127,-118.1488,660,2000-01,0,17,1.0,2PT Field Goal,Right Side Center(RC),Mid-Range,16-24 ft.,2000-11-01,LAL vs. UTA,UTA


In [10]:
dfComplete['total_seconds_remaining'] = dfComplete.seconds_remaining + dfComplete.minutes_remaining
dfComplete

,action_type,combined_shot_type,lat,loc_x,loc_y,lon,minutes_remaining,season,seconds_remaining,shot_distance,shot_made_flag,shot_type,shot_zone_area,shot_zone_basic,shot_zone_range,game_date,matchup,opponent,total_seconds_remaining
1,Jump Shot,Jump Shot,34.0443,-157,0,-118.4268,600,2000-01,22,15,0.0,2PT Field Goal,Left Side(L),Mid-Range,8-16 ft.,2000-10-31,LAL @ POR,POR,622
2,Jump Shot,Jump Shot,33.9093,-101,135,-118.3708,420,2000-01,45,16,1.0,2PT Field Goal,Left Side Center(LC),Mid-Range,16-24 ft.,2000-10-31,LAL @ POR,POR,465
3,Jump Shot,Jump Shot,33.8693,138,175,-118.1318,360,2000-01,52,22,0.0,2PT Field Goal,Right Side Center(RC),Mid-Range,16-24 ft.,2000-10-31,LAL @ POR,POR,412
4,Driving Dunk Shot,Dunk,34.0443,0,0,-118.2698,360,2000-01,19,0,1.0,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,2000-10-31,LAL @ POR,POR,379
5,Jump Shot,Jump Shot,34.0553,-145,-11,-118.4148,540,2000-01,32,14,0.0,2PT Field Goal,Left Side(L),Mid-Range,8-16 ft.,2000-10-31,LAL @ POR,POR,572
6,Layup Shot,Layup,34.0443,0,0,-118.2698,480,2000-01,52,0,1.0,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,2000-10-31,LAL @ POR,POR,532
8,Jump Shot,Jump Shot,33.9363,-65,108,-118.3348,360,2000-01,12,12,1.0,2PT Field Goal,Left Side(L),In The Paint (Non-RA),8-16 ft.,2000-10-31,LAL @ POR,POR,372
9,Running Jump Shot,Jump Shot,33.9193,-33,125,-118.3028,180,2000-01,36,12,0.0,2PT Field Goal,Center(C),In The Paint (Non-RA),8-16 ft.,2000-10-31,LAL @ POR,POR,216
10,Jump Shot,Jump Shot,33.8063,-94,238,-118.3638,60,2000-01,56,25,0.0,3PT Field Goal,Left Side Center(LC),Above the Break 3,24+ ft.,2000-10-31,LAL @ POR,POR,116
11,Jump Shot,Jump Shot,33.9173,121,127,-118.1488,660,2000-01,0,17,1.0,2PT Field Goal,Right Side Center(RC),Mid-Range,16-24 ft.,2000-11-01,LAL vs. UTA,UTA,660


In [11]:
dfComplete.drop(columns=['minutes_remaining', 'seconds_remaining'],inplace=True)

In [12]:
nonScore, score = dfComplete.groupby('shot_made_flag').size()
print(f'Das {dfComplete.shape[0]} entradas no dataset, {nonScore} erraram,  e {score} foram caso onde houve cesta')
print(f'Temos assim {round((score/nonScore) * 100,6)}% de ocorrencias em que o resultado desejamos prever')

Das 25697 entradas no dataset, 14232 erraram,  e 11465 foram caso onde houve cesta
Temos assim 80.557898% de ocorrencias em que o resultado desejamos prever


Verificando outras colunas com atributos faltantes

In [13]:
def getMissingAttributes(dfInput):
    atributos_missing = []
    return_missing = []

    for f in dfInput.columns:
        missings = dfInput[dfInput[f] == -1][f].count()
        if missings > 0:
            atributos_missing.append(f)
            missings_perc = missings/dfInput.shape[0]
            
            return_missing.append([f, missings, missings_perc])

            print('Atributo {} tem {} amostras ({:.2%}) com valores faltantes'.format(f, missings, missings_perc))
            

    print('No total, há {} atributos com valores faltantes'.format(len(atributos_missing)))
    
    return pd.DataFrame(return_missing).rename(index=str, columns={0: "column_name", 1: "column_nulls", 2: "column_percentage"})
missing_Train = getMissingAttributes(dfComplete)
#display(missing_Train)

Atributo loc_x tem 72 amostras (0.28%) com valores faltantes
Atributo loc_y tem 94 amostras (0.37%) com valores faltantes
No total, há 2 atributos com valores faltantes


,column_name,column_nulls,column_percentage
0,loc_x,72,0.002802
1,loc_y,94,0.003658


## Observação
Com base nas analises acima, podemos perceber algumas coisas:

1 -  Os valores estão desnormalizados, variando tanto de tipo (numéricos discretos e continuos, categoricos e binários) como de range de valores;

2 - O unico atributo com valores faltantes significantemente eh o alvo, shot_made_flag, que foram removidos para a competicao. os outros dois tem poucos valores faltando, entao nao vejo muito problema com eles

3 - algumas colunas aparentam dados "repetidos", como lat-lon e loc_x-loc_y. Estes dados podem afetar pouco o resultado, porem manterei eles para analisar os resultados depois

---

In [14]:
def generateMetadata(dfInput):
    data = []
    level = ''
    for f in dfInput.columns:
        # definindo o uso (entre rótulo, id e atributos)
        if f == 'target':
            role = 'target' # rótulo
        elif f == 'id':
            role = 'id'
        else:
            role = 'input' # atributos

        # definindo o tipo do dado
        if 'bin' in f or f == 'target':
            level = 'binary'
        elif 'cat' in f or f == 'id':
            level = 'nominal'
        elif dfInput[f].dtype == float or dfInput[f].dtype == np.float64:
            level = 'interval'
        elif dfInput[f].dtype == int or dfInput[f].dtype == np.int64:
            level = 'ordinal'
            
        # mantem keep como verdadeiro pra tudo, exceto id
        keep = True
        if f == 'id':
            keep = False

        # cria o tipo de dado
        dtype = dfInput[f].dtype

        # cria dicionário de metadados
        f_dict = {
            'varname': f,
            'role': role,
            'level': level,
            'keep': keep,
            'dtype': dtype
        }
        data.append(f_dict)

    meta = pd.DataFrame(data, columns=['varname', 'role', 'level', 'keep', 'dtype'])
    meta.set_index('varname', inplace=True)
    
    return meta

In [15]:
# Models
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

# Feature Selection
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, cross_val_score, ShuffleSplit

# Auxiliary Scores
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

In [16]:
def showDistribution(val_classes):
    nonUsed, used = pd.DataFrame(val_classes).groupby('shot_made_flag').size()
    print('---')
    print(f'Das {pd.DataFrame(val_classes).shape[0]} entradas no dataset, {nonUsed} foram cestas')
    print(f'Temos assim {round((used/len(val_classes)) * 100,6)}% de cestas que desejamos prever')
    print('---')

In [17]:
def logisticRegression(X_Train, y_Train, X_Val, y_Val):

    model = LogisticRegression(solver='lbfgs')

    model.fit(X_Train, y_Train)

    y_pred_class = model.predict(X_Val)
    y_pred_proba = model.predict_proba(X_Val)

    recall = recall_score(y_Val, y_pred_class)
    accuracy = accuracy_score(y_Val, y_pred_class)
    logloss = log_loss(y_Val, y_pred_proba)
    precision =  precision_score(y_Val, y_pred_class)
    f1 = f1_score(y_Val, y_pred_class)
    gini = eval_gini(y_Val, y_pred_class)

    print(f'Baseline - Regressão Logistica')
    print('---')
    print(f'Acurácia: {round(accuracy, 6)}%')
    print(f'Recall: {round(recall, 6)}%')
    print(f'Precisão: {round(precision, 6)}%')
    print(f'Log Loss: {round(logloss, 6)}')
    print(f'F1 Score: {round(f1, 6)}')
    print(f'Gini: {round(gini, 6)}')

    print('---')
    print('Matriz de Confusão')
    #display(pd.DataFrame(confusion_matrix(y_Val, y_pred_class)))
    print('---')
    
    return model, 'Baseline - Regressão Logistica'

In [18]:
# n_estimators=20, learning_rate = 0.5, max_features=2, max_depth = 70, random_state = 0 - Recall: 0.057621%

def xGBClassifier(X_Train, y_Train, X_Val, y_Val, modelName, modelParams):

    if (modelParams == None):
        clf = XGBClassifier()
    else:
        clf = XGBClassifier(**modelParams)  
        modelName = modelName + ' - Parameters: ' + str(modelParams)
    
    clf.fit(X_Train, y_Train)

    y_pred_class = clf.predict(X_Val)
    y_pred_proba = clf.predict_proba(X_Val)

    recall = recall_score(y_Val, y_pred_class)
    accuracy = accuracy_score(y_Val, y_pred_class)
    logloss = log_loss(y_Val, y_pred_proba)
    gini = eval_gini(y_Val, y_pred_class)
    precision =  precision_score(y_Val, y_pred_class)
    f1 = f1_score(y_Val, y_pred_class)

    print(modelName)
    print('---')
    print(f'Acurácia: {round(accuracy, 6)}%')
    print(f'Recall: {round(recall, 6)}%')
    print(f'Precisão: {round(precision, 6)}%')
    print(f'Log Loss: {round(logloss, 6)}')
    print(f'F1 Score: {round(f1, 6)}')
    print(f'Gini: {round(gini, 6)}')

    print('---')
    print('Matriz de Confusão')
    #display(pd.DataFrame(confusion_matrix(y_Val, y_pred_class)))
    print('---')
    
    return clf, modelName

In [19]:
def decisionTreeClassifier(X_Train, y_Train, X_Val, y_Val):

    clf = DecisionTreeClassifier()

    clf.fit(X_Train, y_Train)

    y_pred_class = clf.predict(X_Val)
    y_pred_proba = clf.predict_proba(X_Val)

    recall = recall_score(y_Val, y_pred_class)
    accuracy = accuracy_score(y_Val, y_pred_class)
    gini = eval_gini(y_Val, y_pred_class)
    logloss = log_loss(y_Val, y_pred_proba)
    precision =  precision_score(y_Val, y_pred_class)
    f1 = f1_score(y_Val, y_pred_class)

    print(f'Decision Tree - Default Parameters')
    print('---')
    print(f'Acurácia: {round(accuracy, 6)}%')
    print(f'Recall: {round(recall, 6)}%')
    print(f'Precisão: {round(precision, 6)}%')
    print(f'Log Loss: {round(logloss, 6)}')
    print(f'F1 Score: {round(f1, 6)}')
    print(f'Gini: {round(gini, 6)}')

    print('---')
    print('Matriz de Confusão')
    #display(pd.DataFrame(confusion_matrix(y_Val, y_pred_class)))
    print('---')
    
    return clf, f'Decision Tree - Default Parameters'

In [20]:
def gridSearchKNN(X_Train, y_Train, X_Val, y_Val, k_range):
    clf=KNeighborsClassifier()
    param_grid=dict(n_neighbors=k_range)
    scores = ['f1']
    for sc in scores:
        grid=GridSearchCV(clf,param_grid,cv=4,scoring=sc,n_jobs=-1)
        print("K-Nearest Neighbors - Tuning hyper-parameters for %s" % sc)
        
        grid.fit(X_Train,y_Train)
        
        print(grid.best_params_)
        print(np.round(grid.best_score_,3))
        
        y_pred_class = grid.predict(X_Val)
        y_pred_proba = grid.predict_proba(X_Val)

        recall = recall_score(y_Val, y_pred_class)
        accuracy = accuracy_score(y_Val, y_pred_class)
        gini = eval_gini(y_Val, y_pred_class)
        logloss = log_loss(y_Val, y_pred_proba)
        precision =  precision_score(y_Val, y_pred_class)
        f1 = f1_score(y_Val, y_pred_class)

        print(f'KNN with recall-maxing hyperparameters - {grid.best_params_}')
        print('---')
        print(f'Acurácia: {round(accuracy, 6)}%')
        print(f'Recall: {round(recall, 6)}%')
        print(f'Precisão: {round(precision, 6)}%')
        print(f'Log Loss: {round(logloss, 6)}')
        print(f'F1 Score: {round(f1, 6)}')
        print(f'Gini: {round(gini, 6)}')

        print('---')
        print('Matriz de Confusão')
        #display(pd.DataFrame(confusion_matrix(y_Val, y_pred_class)))
        print('---')
        
        return grid, f'KNN with recall-maxing hyperparameters - {grid.best_params_}'

In [21]:
def gridSearchSVC(X_Train, y_Train, X_Val, y_Val):
    svc=SVC()
    param_grid = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4, 1e-5],'C': [1, 10, 100, 1000]},
                  {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
    scores = ['f1']
    for sc in scores:
        grid=GridSearchCV(svc,param_grid,cv=4,scoring=sc,n_jobs=-1)
        
        print("Support Vector Classifier - Tuning hyper-parameters for %s" % sc)
        
        grid.fit(X_Train,y_Train)
        print(grid.best_params_)
        print(np.round(grid.best_score_,3))
        
        y_pred_class = grid.predict(X_Val)

        recall = recall_score(y_Val, y_pred_class)
        accuracy = accuracy_score(y_Val, y_pred_class)
        gini = eval_gini(y_Val, y_pred_class)
        precision =  precision_score(y_Val, y_pred_class)
        f1 = f1_score(y_Val, y_pred_class)

        print(f'SVC with recall-maxing hyperparameters - {grid.best_params_}')
        print('---')
        print(f'Acurácia: {round(accuracy, 6)}%')
        print(f'Recall: {round(recall, 6)}%')
        print(f'Precisão: {round(precision, 6)}%')
        print(f'F1 Score: {round(f1, 6)}')
        print(f'Gini: {round(gini, 6)}')

        print('---')
        print('Matriz de Confusão')
        #display(pd.DataFrame(confusion_matrix(y_Val, y_pred_class)))
        print('---')
        
        return grid, f'SVC with recall-maxing hyperparameters - {grid.best_params_}'

In [22]:
def gridSearchXGB(X_Train, y_Train, X_Val, y_Val, score):
    xgb=XGBClassifier(random_state = 0)
    param_grid = [{'n_estimators': [100, 200, 300, 400], 'learning_rate': [0.1, 0.25, 0.5, 0.75],'max_depth': [25, 50, 75, 100], 'gamma': [0, 3, 6, 9]}]
    scores = [score]
    for sc in scores:
        grid=GridSearchCV(xgb,param_grid,cv=2,scoring=sc,n_jobs=-1)
        
        print("XGBoost - Tuning hyper-parameters for %s" % sc)
        
        grid.fit(X_Train,y_Train)
        print(grid.best_params_)
        print(np.round(grid.best_score_,3))
        
        y_pred_class = grid.predict(X_Val)

        recall = recall_score(y_Val, y_pred_class)
        accuracy = accuracy_score(y_Val, y_pred_class)
        gini = eval_gini(y_Val, y_pred_class)
        precision =  precision_score(y_Val, y_pred_class)
        f1 = f1_score(y_Val, y_pred_class)

        print(f'XGBoost with {sc}-maxing hyperparameters - {grid.best_params_}')
        print('---')
        print(f'Acurácia: {round(accuracy, 6)}%')
        print(f'Recall: {round(recall, 6)}%')
        print(f'Precisão: {round(precision, 6)}%')
        print(f'F1 Score: {round(f1, 6)}')
        print(f'Gini: {round(gini, 6)}')

        print('---')
        print('Matriz de Confusão')
        #display(pd.DataFrame(confusion_matrix(y_Val, y_pred_class)))
        print('---')
        
        return grid, f'XGBoost with {sc}-maxing hyperparameters - {grid.best_params_}'

In [23]:
def predictTestDataset(X_Test, y_Test, clfModel, clfName):
    y_pred_class = clfModel.predict(X_Test)
    y_pred_proba = clfModel.predict_proba(X_Test)

    recall = recall_score(y_Test, y_pred_class)
    accuracy = accuracy_score(y_Test, y_pred_class)
    gini = eval_gini(y_Test, y_pred_class)
    logloss = log_loss(y_Test, y_pred_proba)
    precision =  precision_score(y_Test, y_pred_class)
    f1 = f1_score(y_Test, y_pred_class)

    print(clfName)
    print('---')
    print(f'Acurácia: {round(accuracy, 6)}%')
    print(f'Recall: {round(recall, 6)}%')
    print(f'Precisão: {round(precision, 6)}%')
    print(f'Log Loss: {round(logloss, 6)}')
    print(f'F1 Score: {round(f1, 6)}')
    print(f'Gini: {round(gini, 6)}')

    print('---')
    print('Matriz de Confusão')
    #display(pd.DataFrame(confusion_matrix(y_Test, y_pred_class)))
    print('---')

In [24]:
def predictContestDataset(X_Test, clfModel, clfName):
    
    print(clfName)
    print('---')
    
    y_pred_class = clfModel.predict(X_Test)
    y_pred_proba = clfModel.predict_proba(X_Test)
    
    pd_prediction = pd.DataFrame(y_pred_class)
    pd_prediction.columns = ['dfComplete']
    showDistribution(pd_prediction)

    return y_pred_class, y_pred_proba

In [25]:
def performOneHotEncoding(dfTrain, dfTest, meta_generic, dist_limit):
    v = meta_generic[(meta_generic.level == 'nominal') & (meta_generic.keep)].index
    #display(v)
    for f in v:
        dist_values = dfTrain[f].value_counts().shape[0]
        print('Atributo {} tem {} valores distintos'.format(f, dist_values))
        if (dist_values > dist_limit):
            print('Atributo {} tem mais de {} valores distintos e por isso será ignorado'.format(f, dist_limit))
            dfTrain.drop([f], axis=1)
            v = v.drop([f])
        
    print('Antes do one-hot encoding tinha-se {} atributos'.format(dfTrain.shape[1]))
    dfTrain = pd.get_dummies(dfTrain, columns=v, drop_first=True)
    print('Depois do one-hot encoding tem-se {} atributos'.format(dfTrain.shape[1]))

    dfTest = pd.get_dummies(dfTest, columns=v, drop_first=True)
    missing_cols = set( dfTrain.columns ) - set( dfTest.columns )
    for c in missing_cols:
        dfTest[c] = 0

    dfTrain, dfTest = dfTrain.align(dfTest, axis=1)
    
    return dfTrain, dfTest

Para analisar os dados, normalizarei alguns valores da tabela. aqui vao as tabelas com valores trocados
1. action_type
2. combined_shot_type
3. shot_type
4. shot_zone_area
5. shot_zone_basic
6. shot_zone_range
7. opponent
8. matchup

Adicionalmente, as columas game_date e season serao removidas, pois nao acredito que influenciem no resultado

In [26]:
dfReplaced = dfComplete
dfReplaced.action_type = pd.factorize(dfComplete.action_type)[0] + 1
dfReplaced.combined_shot_type = pd.factorize(dfComplete.combined_shot_type)[0] + 1
dfReplaced.shot_type = pd.factorize(dfComplete.shot_type)[0] + 1
dfReplaced.shot_zone_basic = pd.factorize(dfComplete.shot_zone_basic)[0] + 1
dfReplaced.shot_zone_area = pd.factorize(dfComplete.shot_zone_area)[0] + 1
dfReplaced.shot_zone_range = pd.factorize(dfComplete.shot_zone_range)[0] + 1
dfReplaced.opponent = pd.factorize(dfComplete.opponent)[0] + 1
dfReplaced.matchup = pd.factorize(dfComplete.matchup)[0] + 1
dfReplaced = dfReplaced.drop(columns=['season','game_date'])
dfReplaced

,action_type,combined_shot_type,lat,loc_x,loc_y,lon,shot_distance,shot_made_flag,shot_type,shot_zone_area,shot_zone_basic,shot_zone_range,matchup,opponent,total_seconds_remaining
1,1,1,34.0443,-157,0,-118.4268,15,0.0,1,1,1,1,1,1,622
2,1,1,33.9093,-101,135,-118.3708,16,1.0,1,2,1,2,1,1,465
3,1,1,33.8693,138,175,-118.1318,22,0.0,1,3,1,2,1,1,412
4,2,2,34.0443,0,0,-118.2698,0,1.0,1,4,2,3,1,1,379
5,1,1,34.0553,-145,-11,-118.4148,14,0.0,1,1,1,1,1,1,572
6,3,3,34.0443,0,0,-118.2698,0,1.0,1,4,2,3,1,1,532
8,1,1,33.9363,-65,108,-118.3348,12,1.0,1,1,3,1,1,1,372
9,4,1,33.9193,-33,125,-118.3028,12,0.0,1,4,3,1,1,1,216
10,1,1,33.8063,-94,238,-118.3638,25,0.0,2,2,4,4,1,1,116
11,1,1,33.9173,121,127,-118.1488,17,1.0,1,3,1,2,2,2,660


splitando em teste e treino

In [27]:
X = dfReplaced.drop(['shot_made_flag'], axis=1)
y = dfReplaced['shot_made_flag']

In [28]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y)

Iniciando treinamento e analise

In [29]:
meta_train = generateMetadata(X_train)
meta_test = generateMetadata(X_val)
#display(meta_train)
#display(meta_test)

,role,level,keep,dtype
varname,,,,
action_type,input,ordinal,True,int64
combined_shot_type,input,binary,True,int64
lat,input,interval,True,float64
loc_x,input,ordinal,True,int64
loc_y,input,ordinal,True,int64
lon,input,interval,True,float64
shot_distance,input,ordinal,True,int64
shot_type,input,ordinal,True,int64
shot_zone_area,input,ordinal,True,int64


,role,level,keep,dtype
varname,,,,
action_type,input,ordinal,True,int64
combined_shot_type,input,binary,True,int64
lat,input,interval,True,float64
loc_x,input,ordinal,True,int64
loc_y,input,ordinal,True,int64
lon,input,interval,True,float64
shot_distance,input,ordinal,True,int64
shot_type,input,ordinal,True,int64
shot_zone_area,input,ordinal,True,int64


In [30]:
print('Tipos e quantidade de features do dataset de treino')
#display(pd.DataFrame({'count' : meta_train.groupby(['role', 'level'])['role'].size()}).reset_index())

print('Tipos e quantidade de features do dataset de teste')
#display(pd.DataFrame({'count' : meta_test.groupby(['role', 'level'])['role'].size()}).reset_index())

Tipos e quantidade de features do dataset de treino


,role,level,count
0,input,binary,1
1,input,interval,2
2,input,ordinal,11


Tipos e quantidade de features do dataset de teste


,role,level,count
0,input,binary,1
1,input,interval,2
2,input,ordinal,11


In [31]:
X_train_one, X_test_one = performOneHotEncoding(X_train, y_train, meta_train, 200)

Index([], dtype='object', name='varname')

Antes do one-hot encoding tinha-se 14 atributos
Depois do one-hot encoding tem-se 14 atributos


In [32]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)
    
showDistribution(y_train)
logRegModel, logRegName = logisticRegression(X_train, y_train, X_val, y_val)
xgbPureModel, xgbPureName = xGBClassifier(X_train, y_train, X_val, y_val, 'XGBoost - Base',None)
#xgbPresetModel, xgbPresetName = xGBClassifier(X_train, y_train, X_val, y_val, 'XGBoost - Preset', {'n_estimator':400, 'learning_rate' : 0.5,'random_state' : 0,'max_depth':70,'objective':"binary:logistic",'subsample':.8,'min_child_weig':6,'colsample_bytr':.8,'scale_pos_weight':1.6, 'gamma':10, 'reg_alph':8, 'reg_lambda':1})
xgbHyperParametrizedModel, xgbHyperParametrizedName = xGBClassifier(X_train, y_train, X_val, y_val, 'XGBoost - Hyperparametrized',{'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 2, 'min_child_weight': 16, 'n_estimators': 100, 'subsample': 1.0})
#decTreeModel, decTreeName = decisionTreeClassifier(X_train, y_train, X_val, y_val)
#knnModel, knnName = gridSearchKNN(X_train, y_train, X_val, y_val, list(range(1,20)))
#svcModel, svcName = gridSearchSVC(X_train, y_train, X_val, y_val)
    
showDistribution(y)
predictTestDataset(X, y, logRegModel, logRegName)
predictTestDataset(X, y, xgbPureModel, xgbPureName)
#predictTestDataset(X_supersampled, y_supersampled, xgbPresetModel, xgbPresetName)
predictTestDataset(X, y, xgbHyperParametrizedModel, xgbHyperParametrizedName)
#predictTestDataset(X_supersampled, y_supersampled, decTreeModel, decTreeName)
#predictTestDataset(X_supersampled, y_supersampled, knnModel, knnName)
#predictTestDataset(X_supersampled, y_supersampled, svcModel, svcName)

---
Das 17987 entradas no dataset, 9962 foram cestas
Temos assim 44.615556% de cestas que desejamos prever
---
Baseline - Regressão Logistica
---
Acurácia: 0.633333%
Recall: 0.466279%
Precisão: 0.618112%
Log Loss: 0.655592
F1 Score: 0.531566
Gini: 0.229148
---
Matriz de Confusão


,0,1
0,3279,991
1,1836,1604


---
XGBoost - Base
---
Acurácia: 0.673411%
Recall: 0.456977%
Precisão: 0.707471%
Log Loss: 0.616611
F1 Score: 0.555281
Gini: 0.302701
---
Matriz de Confusão


,0,1
0,3620,650
1,1868,1572


---
XGBoost - Hyperparametrized - Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 2, 'min_child_weight': 16, 'n_estimators': 100, 'subsample': 1.0}
---
Acurácia: 0.6738%
Recall: 0.456395%
Precisão: 0.708804%
Log Loss: 0.617664
F1 Score: 0.555261
Gini: 0.3051
---
Matriz de Confusão


...

---
XGBoost - Base
---
Acurácia: 0.682531%
Recall: 0.471173%
Precisão: 0.720555%
Log Loss: 0.60189
F1 Score: 0.569771
Gini: 0.330335
---
Matriz de Confusão


,0,1
0,12137,2095
1,6063,5402


---
XGBoost - Hyperparametrized - Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 2, 'min_child_weight': 16, 'n_estimators': 100, 'subsample': 1.0}
---
Acurácia: 0.681325%
Recall: 0.468295%
Precisão: 0.719512%
Log Loss: 0.604671
F1 Score: 0.567338
Gini: 0.326421
---
Matriz de Confusão


,0,1
0,12139,2093
1,6096,5369


---


In [33]:
showDistribution(y_train)
xgbGridSearchModel, xgbGridSearchName = gridSearchXGB(X_train, y_train, X_val, y_val, 'f1')

---
Das 17987 entradas no dataset, 9962 foram cestas
Temos assim 44.615556% de cestas que desejamos prever
---
XGBoost - Tuning hyper-parameters for f1
{'gamma': 6, 'learning_rate': 0.1, 'max_depth': 75, 'n_estimators': 100}
0.576
XGBoost with f1-maxing hyperparameters - {'gamma': 6, 'learning_rate': 0.1, 'max_depth': 75, 'n_estimators': 100}
---
Acurácia: 0.649416%
Recall: 0.487209%
Precisão: 0.640918%
F1 Score: 0.553592
Gini: 0.268203
---
Matriz de Confusão


,0,1
0,3331,939
1,1764,1676


---


E finalmente utilizando XGB

In [34]:
showDistribution(y)
predictTestDataset(X, y, xgbGridSearchModel, xgbGridSearchName)

---
Das 25697 entradas no dataset, 14232 foram cestas
Temos assim 44.616103% de cestas que desejamos prever
---
XGBoost with f1-maxing hyperparameters - {'gamma': 6, 'learning_rate': 0.1, 'max_depth': 75, 'n_estimators': 100}
---
Acurácia: 0.767327%
Recall: 0.643611%
Precisão: 0.795837%
Log Loss: 0.498279
F1 Score: 0.711675
Gini: 0.513414
---
Matriz de Confusão


,0,1
0,12339,1893
1,4086,7379


---


In [35]:
X

,action_type,combined_shot_type,lat,loc_x,loc_y,lon,shot_distance,shot_type,shot_zone_area,shot_zone_basic,shot_zone_range,matchup,opponent,total_seconds_remaining
1,1,1,34.0443,-157,0,-118.4268,15,1,1,1,1,1,1,622
2,1,1,33.9093,-101,135,-118.3708,16,1,2,1,2,1,1,465
3,1,1,33.8693,138,175,-118.1318,22,1,3,1,2,1,1,412
4,2,2,34.0443,0,0,-118.2698,0,1,4,2,3,1,1,379
5,1,1,34.0553,-145,-11,-118.4148,14,1,1,1,1,1,1,572
6,3,3,34.0443,0,0,-118.2698,0,1,4,2,3,1,1,532
8,1,1,33.9363,-65,108,-118.3348,12,1,1,3,1,1,1,372
9,4,1,33.9193,-33,125,-118.3028,12,1,4,3,1,1,1,216
10,1,1,33.8063,-94,238,-118.3638,25,2,2,4,4,1,1,116
11,1,1,33.9173,121,127,-118.1488,17,1,3,1,2,2,2,660


In [36]:
X = X.drop(columns=['lat','lon','matchup','opponent'])

In [37]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)
    
showDistribution(y_train)
logRegModel, logRegName = logisticRegression(X_train, y_train, X_val, y_val)
xgbPureModel, xgbPureName = xGBClassifier(X_train, y_train, X_val, y_val, 'XGBoost - Base',None)
#xgbPresetModel, xgbPresetName = xGBClassifier(X_train, y_train, X_val, y_val, 'XGBoost - Preset', {'n_estimator':400, 'learning_rate' : 0.5,'random_state' : 0,'max_depth':70,'objective':"binary:logistic",'subsample':.8,'min_child_weig':6,'colsample_bytr':.8,'scale_pos_weight':1.6, 'gamma':10, 'reg_alph':8, 'reg_lambda':1})
xgbHyperParametrizedModel, xgbHyperParametrizedName = xGBClassifier(X_train, y_train, X_val, y_val, 'XGBoost - Hyperparametrized',{'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 2, 'min_child_weight': 16, 'n_estimators': 100, 'subsample': 1.0})
#decTreeModel, decTreeName = decisionTreeClassifier(X_train, y_train, X_val, y_val)
#knnModel, knnName = gridSearchKNN(X_train, y_train, X_val, y_val, list(range(1,20)))
#svcModel, svcName = gridSearchSVC(X_train, y_train, X_val, y_val)
    
showDistribution(y)
predictTestDataset(X, y, logRegModel, logRegName)
predictTestDataset(X, y, xgbPureModel, xgbPureName)
#predictTestDataset(X_supersampled, y_supersampled, xgbPresetModel, xgbPresetName)
predictTestDataset(X, y, xgbHyperParametrizedModel, xgbHyperParametrizedName)
#predictTestDataset(X_supersampled, y_supersampled, decTreeModel, decTreeName)
#predictTestDataset(X_supersampled, y_supersampled, knnModel, knnName)
#predictTestDataset(X_supersampled, y_supersampled, svcModel, svcName)

---
Das 17987 entradas no dataset, 9962 foram cestas
Temos assim 44.615556% de cestas que desejamos prever
---
Baseline - Regressão Logistica
---
Acurácia: 0.629702%
Recall: 0.459884%
Precisão: 0.613416%
Log Loss: 0.655639
F1 Score: 0.525669
Gini: 0.22246
---
Matriz de Confusão


,0,1
0,3273,997
1,1858,1582


---
XGBoost - Base
---
Acurácia: 0.672763%
Recall: 0.456105%
Precisão: 0.706439%
Log Loss: 0.616873
F1 Score: 0.554319
Gini: 0.301103
---
Matriz de Confusão


,0,1
0,3618,652
1,1871,1569


---
XGBoost - Hyperparametrized - Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 2, 'min_child_weight': 16, 'n_estimators': 100, 'subsample': 1.0}
---
Acurácia: 0.672244%
Recall: 0.455523%
Precisão: 0.705538%
Log Loss: 0.618515
F1 Score: 0.553612
Gini: 0.301129
---
Matriz de Confusão


...

---
XGBoost - Base
---
Acurácia: 0.682414%
Recall: 0.470563%
Precisão: 0.720679%
Log Loss: 0.602829
F1 Score: 0.569363
Gini: 0.329406
---
Matriz de Confusão


,0,1
0,12141,2091
1,6070,5395


---
XGBoost - Hyperparametrized - Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 2, 'min_child_weight': 16, 'n_estimators': 100, 'subsample': 1.0}
---
Acurácia: 0.681208%
Recall: 0.469254%
Precisão: 0.718579%
Log Loss: 0.605523
F1 Score: 0.56775
Gini: 0.326171
---
Matriz de Confusão


,0,1
0,12125,2107
1,6085,5380


---


#### No fim, varias analises obtiveram resultados positivos com relação à acurácia, como regressão (67% após XGB), e XGB com f1-max, que apresentou 76%, porém isso é devido a overfitting no segundo XGB. 

#### Analisando o resultado sem outros campos, como lat, lon, opponent e matchup, os resultados permaneceram praticamente inalterados, mostrando que minha hipotese de que eles não influenciavam na analise de cesta ou não estava correta. O mesmo acontece se remover loc_x e loc_y e manter lat-lon. 

#### Por fim, seria interessante comparar os dados dessa análise com dados reais do estado fisico do jogador no dia do jogo, podendo haver alguma influencia externa na capacidade de fazer pontos